In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
from alibi_detect.cd import CVMDrift, MMDDrift, FETDrift

Importing plotly failed. Interactive plots will not work.


# Continuous data

In [2]:
ntimes = 1
times = np.empty([ntimes])

p_val = 0.05
N = 1000 # Size of reference set

# Note how these are diff to the Gaussian used for configuration.
x_ref = np.random.uniform(0, 1, N).reshape(-1,1)
stream_h0 = np.random.uniform(size=500)
stream_h1 = np.random.exponential(size=500)

for t in range(ntimes):
    t0 = time.time()
    ddcvm = CVMDrift(x_ref, p_val=0.05)
    times[t] = time.time() - t0
    ddmmd = MMDDrift(x_ref, p_val=0.05)

print('Wall time = %.3fs +- %.2gs' % (times.mean(),times.std()))

Wall time = 0.000s +- 0s


In [3]:
ddmmd.predict(stream_h0)

{'data': {'is_drift': 0,
  'distance': -0.00026494265,
  'p_val': 0.49,
  'threshold': 0.05,
  'distance_threshold': 0.003301859},
 'meta': {'name': 'MMDDriftTF',
  'detector_type': 'offline',
  'data_type': None,
  'version': '0.7.3dev',
  'backend': 'tensorflow'}}

In [4]:
ddmmd.predict(stream_h1)

{'data': {'is_drift': 1,
  'distance': 0.08218652,
  'p_val': 0.0,
  'threshold': 0.05,
  'distance_threshold': 0.0029037595},
 'meta': {'name': 'MMDDriftTF',
  'detector_type': 'offline',
  'data_type': None,
  'version': '0.7.3dev',
  'backend': 'tensorflow'}}

In [5]:
ddcvm.predict(stream_h0)

{'data': {'is_drift': 0,
  'distance': 0.09414622222223556,
  'p_val': 0.6150868355205165,
  'threshold': 0.05},
 'meta': {'name': 'CVMDrift',
  'detector_type': 'offline',
  'data_type': None,
  'version': '0.7.3dev'}}

In [6]:
ddcvm.predict(stream_h1)

{'data': {'is_drift': 1,
  'distance': 7.592496888888888,
  'p_val': 8.981260180007666e-10,
  'threshold': 0.05},
 'meta': {'name': 'CVMDrift',
  'detector_type': 'offline',
  'data_type': None,
  'version': '0.7.3dev'}}

# Categorical data

In [24]:
N = 5000
p_h0 = 0.5
p_h1 = 0.4
x_ref = np.random.choice(2, N, p=[1-p_h0, p_h0])
x_test0 = np.random.choice(2, 1000, p=[1-p_h0, p_h0]) 
x_test1 = np.random.choice(2, 1000, p=[1-p_h1, p_h1]) 

In [25]:
ddfet = FETDrift(x_ref, p_val = 0.05)

In [26]:
ddfet.predict(x_test0)

{'data': {'is_drift': 0,
  'distance': 0.8952724623493119,
  'p_val': 0.11889575172705508,
  'threshold': 0.05},
 'meta': {'name': 'FETDrift',
  'detector_type': 'offline',
  'data_type': None,
  'version': '0.7.3dev'}}

In [10]:
ddfet.predict(x_test1)

{'data': {'is_drift': 1,
  'distance': 1.27401764673645,
  'p_val': 0.0005230974091784745,
  'threshold': 0.05},
 'meta': {'name': 'FETDrift',
  'detector_type': 'offline',
  'data_type': None,
  'version': '0.7.3dev'}}

In [11]:
x = x_test1
a = np.sum(x_ref)
b = len(x_ref) - a
c = np.sum(x)
d = len(x) - c
table = np.array([[a,b],[c,d]])
print(table)

[[2430 2570]
 [ 426  574]]


In [12]:
from scipy.stats import hypergeom, fisher_exact
oddsratio, p_val = fisher_exact(table, alternative='two-sided')
print(p_val)
#[M, n, N] = [20, 7, 12]
#rv = hypergeom(M, n, N)

0.0005230974091784745


In [13]:
values, counts = np.unique(x, return_counts=True)
values

array([0, 1])

In [14]:
counts

array([574, 426])

In [15]:
z_ref = np.random.choice([False, True], (1,100), p=[0.4,1-0.4])

In [16]:
set(np.unique(z_ref))

{False, True}

In [17]:
set((True, False)) == set(np.unique(z_ref))

True

In [18]:
set(np.unique(x_ref)) != {0,1}

False

In [19]:
set(np.unique(x_ref))

{0, 1}

In [20]:
type({0,1}.pop())

int